In [0]:
import numpy as np
import cv2
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import shutil
import dill
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
from google.colab import drive, files
from torch.utils.data import Dataset, DataLoader, random_split
import librosa, librosa.display
import torch.nn.functional as F

In [0]:
drive.mount('/drive', force_remount=True)

In [0]:
load_path = '/drive/My Drive/composerDataset.dill'
fs = 44100
composers = ['Schubert', 'Beethoven', 'Brahms', 'Mozart', 'Bach']
segment_duration = 20
n_samples = 300
train_test_partition = 0.8


In [0]:
import dill
with open(load_path, 'rb') as file:
  dataset = dill.load(file)


In [0]:
train_len = int(train_test_partition * len(composers) * n_samples)
test_len = len(composers) * n_samples - train_len
train_set, test_set = random_split(dataset, [1200, 300])
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 16, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 16, shuffle = True)

In [0]:
count = [0,0,0,0,0]
for n in range(len(train_set)):
  _,label = train_set[n]
  count[label.item()] =+ 1
count

[1, 1, 1, 1, 1]

In [0]:
dataset[150][0][0][0]

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.get_device_name(0))
print(device)

In [0]:
import torchvision
model = torchvision.models.resnet34(pretrained=False, progress=True)
num_classes = 5
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(512, num_classes)

model = nn.Sequential(
    model,
    nn.Softmax(1)
)

model.train()
model.to(device)


In [0]:
optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)

In [0]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [0]:
log_interval = 20
for epoch in range(1, 5):
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
    scheduler.step()
    train(model, epoch)

In [0]:
def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        correct += pred.eq(target).cpu().sum().item()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
model.eval()
g, id = test_set[100]
g = g.unsqueeze(0).to(device)
print(id.item())
model(g)

In [0]:
fig = plt.figure(figsize=(10,4))
S_dB,_ = dataset[150][0]
plt.pcolormesh(S_dB[0][0], cmap = 'magma')


In [0]:
a, _  = dataset[0]
a[0][0]

In [0]:
count = 0
for i in range(len(train_set)):
  b ,_ = train_set[i]
  if torch.abs(b).sum().item() < 0.001 :
    count += 1
    print('error')

In [0]:
print(np.max)


In [0]:
save_image_path = '/drive/My Drive/Academic/EE473/specDataset.dill'
import dill
with open(save_image_path, 'wb') as file:
  dill.dump(specFigureData, file)

In [0]:
torch.max